In [1]:
import string
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql import types

In [2]:
# create sparksession
ss = SparkSession \
    .builder \
    .appName("Manipulation DataFrame") \
    .master("local[2]") \
    .getOrCreate()

In [3]:
# load data
df = ss.read.json("./data/data.json")

## 数据评估

### 显示数据类型
显示数据类型的方法，可以使用 `printSchema()` 以及 `dtypes` 属性。前者是返回树形结构，后者是得到一个键值形式列表

In [4]:
# 显示 Schema
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- altitude: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- exact_location: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- indoor: long (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- longitude: string (nullable = true)
 |-- sampling_rate: string (nullable = true)
 |-- sensor: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- pin: string (nullable = true)
 |    |-- sensor_type: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- manufacturer: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- sensordatavalues: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- value: string (nullable = true)
 |    |    |-- value_type: string (nullable = true)
 |-- timestamp: string (nullab

In [5]:
df.dtypes

[('id', 'bigint'),
 ('location',
  'struct<altitude:string,country:string,exact_location:bigint,id:bigint,indoor:bigint,latitude:string,longitude:string>'),
 ('sampling_rate', 'string'),
 ('sensor',
  'struct<id:bigint,pin:string,sensor_type:struct<id:bigint,manufacturer:string,name:string>>'),
 ('sensordatavalues',
  'array<struct<id:bigint,value:string,value_type:string>>'),
 ('timestamp', 'string')]

### 显示DataFrame 维度数据

In [6]:
df.count(), len(df.columns)

(35527, 6)

### 显示前几行数据
有多种方法显示前几行数据，`head()`以及特殊的方法 `first()`、`show()`、`limit()` 等。需要注意返回的结果类型不是完全相同的。

In [7]:
df.head(2)

[Row(id=5756852209, location=Row(altitude='104.9', country='UA', exact_location=0, id=22256, indoor=1, latitude='50.51', longitude='30.798'), sampling_rate=None, sensor=Row(id=36214, pin='7', sensor_type=Row(id=9, manufacturer='various', name='DHT22')), sensordatavalues=[Row(id=12224991603, value='10.00', value_type='temperature'), Row(id=12224991604, value='50.70', value_type='humidity')], timestamp='2019-12-13 11:10:02'),
 Row(id=5756852208, location=Row(altitude='111.8', country='GB', exact_location=1, id=21003, indoor=0, latitude='53.87869338867', longitude='-1.45841360092'), sampling_rate=None, sensor=Row(id=34792, pin='11', sensor_type=Row(id=17, manufacturer='Bosch', name='BME280')), sensordatavalues=[Row(id=12224991602, value='96357.16', value_type='pressure'), Row(id=12224991605, value='1.93', value_type='temperature'), Row(id=12224991606, value='100.00', value_type='humidity'), Row(id=None, value='97702.38', value_type='pressure_at_sealevel')], timestamp='2019-12-13 11:10:02'

In [8]:
df.first()

Row(id=5756852209, location=Row(altitude='104.9', country='UA', exact_location=0, id=22256, indoor=1, latitude='50.51', longitude='30.798'), sampling_rate=None, sensor=Row(id=36214, pin='7', sensor_type=Row(id=9, manufacturer='various', name='DHT22')), sensordatavalues=[Row(id=12224991603, value='10.00', value_type='temperature'), Row(id=12224991604, value='50.70', value_type='humidity')], timestamp='2019-12-13 11:10:02')

In [9]:
df.show(n=2)

+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|        id|            location|sampling_rate|              sensor|    sensordatavalues|          timestamp|
+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|5756852209|[104.9, UA, 0, 22...|         null|[36214, 7, [9, va...|[[12224991603, 10...|2019-12-13 11:10:02|
|5756852208|[111.8, GB, 1, 21...|         null|[34792, 11, [17, ...|[[12224991602, 96...|2019-12-13 11:10:02|
+----------+--------------------+-------------+--------------------+--------------------+-------------------+
only showing top 2 rows



### 显示数据列名称
通过显示 `schema` 和 `dtypes` 可以显示列名称，需要直接得到列名称可以直接通过 `columns` 属性获取

In [10]:
df.columns

['id', 'location', 'sampling_rate', 'sensor', 'sensordatavalues', 'timestamp']

### 缺失值统计
DataFrame 没有自带的检测是否为缺失值方法，需要通过调用 `functions` 模块中的方法可以进行统计缺失值，大致步骤如下：
1. 通过 `isnull()` 判断是否为缺失值，此外需要注意该方法和 pandas 有差异——pandas 没有区分 NAN 和 NULL。Spark 支持 [eqNullSafe](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.eqNullSafe) 安全测试
2. 使用 `cast()` 方法将数据类型转换为 `integer`。该步骤可以通过 [`when()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.when)  和 [`otherwise()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.otherwise)方法完成
3. 通过 `sum()` 方法累计求和

In [11]:
# 使用 pandas 的 isnull 方法判断
pd.isnull(np.nan), pd.isnull(None)

(True, True)

In [12]:
df.select(
    functions.sum(
        functions.isnull("sampling_rate").cast("integer")
    ).alias("sampling_rate")
).show()

+-------------+
|sampling_rate|
+-------------+
|        35527|
+-------------+



In [13]:
# 第二种方法，通过 when 和 otherwise 完成——实际内核是 SQL 中 CASE WHEN 方法
# 注意 when 和 otherwise 可以链式表达而不需要额外的
df.select(
    functions.when(
        functions.isnull("sampling_rate"), 1
    ).otherwise(0).alias("sampling_rate")
).select(functions.sum("sampling_rate").alias("Miss_value")).show()

+----------+
|Miss_value|
+----------+
|     35527|
+----------+



In [14]:
# 计算 columns 的缺失值数量
df.select([functions.sum(functions.when(functions.isnull(c), 1).otherwise(0)).alias(c) for c in df.columns]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [15]:
# 替换方法计算 columns 缺失值
df.select(
        [functions.sum(
            functions.isnull(x).cast("integer")
        ).alias(x)
         for x in df.columns
        ]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [16]:
# 通过 UDF 计算，需要先分别检验各列中缺失值，在进行计算
def check_missing(x):
    if x is None:
        return 1
    else:
        return 0
        
missing_count = functions.udf(lambda x: check_missing(x), types.IntegerType())

In [17]:
# 计算单一列
df.select(missing_count("sampling_rate").alias("test")).select(functions.sum("test")).show()

+---------+
|sum(test)|
+---------+
|    35527|
+---------+



In [18]:
df.select([
    missing_count(x).alias(x) for x in df.columns
    ]).select([
        functions.sum(x).alias(x) for x in df.columns
    ]).show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



In [19]:
# 使用 * 可以筛选列，并且可以定制化通配符筛选
df.select([
    missing_count(x).alias(x) for x in df.columns
    ]).select([
        functions.sum(x).alias(x) for x in df.columns
    ]).select("*").show()

+---+--------+-------------+------+----------------+---------+
| id|location|sampling_rate|sensor|sensordatavalues|timestamp|
+---+--------+-------------+------+----------------+---------+
|  0|       0|        35527|     0|               0|        0|
+---+--------+-------------+------+----------------+---------+



### 显示基本的统计信息
pandas 中可以通过 `DataFrame.describe()` 查看各列的统计信息。Spark 也支持该功能，可以使用 `summary()` 和 `describe()` 方法查询

In [20]:
df.describe().show()

+-------+-------------------+-------------+-------------------+
|summary|                 id|sampling_rate|          timestamp|
+-------+-------------------+-------------+-------------------+
|  count|              35527|            0|              35527|
|   mean|5.756834376153911E9|         null|               null|
| stddev| 10295.038530963458|         null|               null|
|    min|         5756816550|         null|2019-12-13 11:04:53|
|    max|         5756852209|         null|2019-12-13 11:10:02|
+-------+-------------------+-------------+-------------------+



In [21]:
df.summary().show()

+-------+-------------------+-------------+-------------------+
|summary|                 id|sampling_rate|          timestamp|
+-------+-------------------+-------------+-------------------+
|  count|              35527|            0|              35527|
|   mean|5.756834376153911E9|         null|               null|
| stddev| 10295.038530963458|         null|               null|
|    min|         5756816550|         null|2019-12-13 11:04:53|
|    25%|         5756825456|         null|               null|
|    50%|         5756834377|         null|               null|
|    75%|         5756843280|         null|               null|
|    max|         5756852209|         null|2019-12-13 11:10:02|
+-------+-------------------+-------------+-------------------+



### 数据冗余统计
Spark 统计数据冗余，和 Pandas 存在差异——pandas 直接提供了 `DataFrame.duplicate()` 方法判断数据是否冗余。Spark 需要通过多步骤完成，[countDistinc](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.countDistinct) 统计唯一值数量——其中非数值数据也是统计为一个唯一值， [count](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.count) 统计数据长度。

当需要以多字段判断是否有冗余值时，需要通过 groupby 计数数量大于 1 的组合。

In [22]:
list(range(20))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [23]:
# 创建数据
schema = types.StructType([
    types.StructField("col1", types.StringType()),
    types.StructField("col2", types.FloatType()),
    types.StructField("col3", types.IntegerType()),
    types.StructField("col4", types.FloatType()),
     types.StructField("col5", types.StringType())
])

test = ss.createDataFrame(
    list(zip(
        np.random.choice(list(string.ascii_uppercase)+[None]*20, size=30).tolist(), 
        np.random.normal(0, 10,  size=30).tolist(), 
        np.random.randint(0, 20,  size=30).tolist(),
        np.random.choice(list(range(4)) + [np.nan], size=30).tolist(),
        np.random.choice(list(string.ascii_uppercase)+[None]*20, size=30).tolist())), schema=schema
)

test.persist()

test.head(2)

[Row(col1='T', col2=-10.2372465133667, col3=18, col4=3.0, col5=None),
 Row(col1='O', col2=1.5313410758972168, col3=9, col4=0.0, col5=None)]

In [24]:
# 统计各列的长度
test.select([functions.count(x).alias(x) for x in test.columns]).show()

+----+----+----+----+----+
|col1|col2|col3|col4|col5|
+----+----+----+----+----+
|  21|  30|  30|  30|  13|
+----+----+----+----+----+



In [25]:
# 该方法将缺失值也是统计为了唯一值
test.select("col4").distinct().show()

+----+
|col4|
+----+
| 2.0|
| 3.0|
| 1.0|
| NaN|
| 0.0|
+----+



In [26]:
# 统计各列唯一值数量
test.select([functions.countDistinct(x).alias(x) for x in test.columns]).show()

+----+----+----+----+----+
|col1|col2|col3|col4|col5|
+----+----+----+----+----+
|  14|  30|  14|   5|  10|
+----+----+----+----+----+



In [27]:
# 通过两个长度值减去唯一值长度计算数据冗余数量
test.select([(functions.count(x) - functions.countDistinct(x)).alias(x) for x in test.columns]).show()

+----+----+----+----+----+
|col1|col2|col3|col4|col5|
+----+----+----+----+----+
|   7|   0|  16|  25|   3|
+----+----+----+----+----+



In [29]:
# 统计各组合下的冗余数量
test.groupBy("col1", "col4").count().filter("count > 1" ).show()

+----+----+-----+
|col1|col4|count|
+----+----+-----+
|null| 3.0|    3|
|   W| 0.0|    2|
|null| NaN|    3|
|   T| 3.0|    2|
+----+----+-----+



## 数据清理

### 类型转换

* 日期或时间戳转换为日期字符串 使用 [date_format()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.date_format) 方法进行转换
* 日期字符串转换转换为日期或时间戳 使用 [to_date()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.to_date) 和 [to_timestamp](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.to_timestamp) 方法进行转换，如果没有设置 `format` 参数前者相当于使用 `col.cast("date")`后者相当于使用了 `col.cast("timestamp")`。需要注意两者在使用的格式需要依据 [Java 模式格式](https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html)
* 比较通用的转换方法是使用 [cast()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast) 进行类型转换——思路和 SQL 中 `cast()` 相似。此外还有类似的 [astype()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.astype)

需要注意第三种方法是 `Column` 的方法，在使用的时候最好强制使用 `df.select(functions.col("datetime").cast("string"))`

In [30]:
# 转换为时间戳
df.select(functions.to_timestamp("timestamp", "yyyy-MM-dd hh:mm:ss").alias("time")).dtypes

[('time', 'timestamp')]

In [31]:
# 时间戳转换为字符串
df.select(
    functions.to_timestamp("timestamp", "yyyy-MM-dd hh:mm:ss").alias("time")
).select(
    functions.date_format("time", "yyyy-MM-dd hh:mm:ss").alias("time")
).dtypes

[('time', 'string')]

In [32]:
# 使用 cast 方法
test.select(functions.col("col2").cast("string")).dtypes

[('col2', 'string')]

In [33]:
# 使用 astype 方法
df.select(functions.col("timestamp").astype("date").alias("date_")).head(2)

[Row(date_=datetime.date(2019, 12, 13)),
 Row(date_=datetime.date(2019, 12, 13))]

### 数据筛选
在 Spark 中进行数据筛选，可以通过 [filter()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter) 或者 [where()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where) 方法配合条件进行筛选。两者是 `DataFrame` 下的方法。

* 非缺失值条件筛选 单列缺失值判断可以直接使用 [isNotNull()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.isNotNull)，多列筛选需要使用  '&' 替换 'and', '|' 替换 'or', '~' 替换 'not' ——但是通过 SQL 表达式时，需要直接使用逻辑运算的原始自符
* 缺失值条件筛选 单列缺失值判断直接使用 [isNull()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.isNull)，在 `filter()` 中可以使用 `<col_name> is NULL` 进行筛选，多列筛选需要使用  '&' 替换 'and', '|' 替换 'or', '~' 替换 'not' ——但是通过 SQL 表达式时，需要直接使用逻辑运算的原始自符
* 其他条件筛选 pandas 中可以通过掩码方式筛选数据，Spark 中需要借助 `filter()` 和 `where()` 构造条件表达式筛选——例如 `df.filter(df.id > 12).show()` 或者 `df.filter("id > 12").show()`

从上面的结果可以看出，构造出可用条件是非常重要的。Spark 的 [Column](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column) 提供了丰富的方法，例如 `endswith()`和 `startswith()`——不支持正则表达式，`isin()`，`like()`——支持类似 SQL 中 like 方式，`rlike()`——支持正则表达式，`contains()`，`between()`—— ` df.select(df.age.between(2, 4)).show()` 类似于使用 `WHERE age BETWEEN 2 AND 4; `



In [34]:
# 使用 where 进行筛选非缺失列
test.where(test.col1.isNotNull()).show()

+----+-----------+----+----+----+
|col1|       col2|col3|col4|col5|
+----+-----------+----+----+----+
|   T|-10.2372465|  18| 3.0|null|
|   O|  1.5313411|   9| 0.0|null|
|   S| -9.1716175|   5| 0.0|   J|
|   X|   5.094148|   3| NaN|   X|
|   E|  -3.525113|   3| 2.0|null|
|   W|  -18.93121|  10| 2.0|   R|
|   T| -7.7867312|   2| 3.0|null|
|   N| -14.736388|   6| 3.0|null|
|   R| -3.1653242|   7| 0.0|null|
|   S|  27.000383|   8| 1.0|null|
|   Z|  12.456934|  19| NaN|   K|
|   B| -3.7770996|  18| 2.0|null|
|   H|   2.598025|   9| NaN|null|
|   M|  16.468563|  17| 2.0|   I|
|   W| -5.6359506|   7| 0.0|   A|
|   B| -13.762591|  19| NaN|   S|
|   F| -0.5584573|  14| 0.0|null|
|   R|  5.4427624|   4| 1.0|   L|
|   W| 0.61386025|  10| 0.0|null|
|   X| -4.4530787|   9| 1.0|   I|
+----+-----------+----+----+----+
only showing top 20 rows



In [35]:
# 使用 filter 筛选非缺失列
test.filter(test.col1.isNotNull()).show()

+----+-----------+----+----+----+
|col1|       col2|col3|col4|col5|
+----+-----------+----+----+----+
|   T|-10.2372465|  18| 3.0|null|
|   O|  1.5313411|   9| 0.0|null|
|   S| -9.1716175|   5| 0.0|   J|
|   X|   5.094148|   3| NaN|   X|
|   E|  -3.525113|   3| 2.0|null|
|   W|  -18.93121|  10| 2.0|   R|
|   T| -7.7867312|   2| 3.0|null|
|   N| -14.736388|   6| 3.0|null|
|   R| -3.1653242|   7| 0.0|null|
|   S|  27.000383|   8| 1.0|null|
|   Z|  12.456934|  19| NaN|   K|
|   B| -3.7770996|  18| 2.0|null|
|   H|   2.598025|   9| NaN|null|
|   M|  16.468563|  17| 2.0|   I|
|   W| -5.6359506|   7| 0.0|   A|
|   B| -13.762591|  19| NaN|   S|
|   F| -0.5584573|  14| 0.0|null|
|   R|  5.4427624|   4| 1.0|   L|
|   W| 0.61386025|  10| 0.0|null|
|   X| -4.4530787|   9| 1.0|   I|
+----+-----------+----+----+----+
only showing top 20 rows



In [36]:
# 条件筛选缺失行， 等价于
# test.where("col1 is NULL").show()
# test.filter(test.col1.isNull()).show()
# test.where(test.col1.isNull()).show()
test.filter("col1 is NULL").show()

+----+----------+----+----+----+
|col1|      col2|col3|col4|col5|
+----+----------+----+----+----+
|null|  7.406317|  19| 1.0|null|
|null| -4.990888|   3| NaN|null|
|null|   8.07055|  15| NaN|null|
|null| 2.4262414|  19| 2.0|   K|
|null|-4.2632003|   2| NaN|   V|
|null|-3.3306575|  10| 0.0|   E|
|null| -16.01117|   8| 3.0|null|
|null|-1.0873488|   8| 3.0|null|
|null| 3.2200346|  18| 3.0|   X|
+----+----------+----+----+----+



In [37]:
# 多列筛选缺失行，等价于
# test.filter("col1 is NULL and col5 is NULL").show()
test.filter(test.col1.isNull() & test.col5.isNull()).show()

+----+----------+----+----+----+
|col1|      col2|col3|col4|col5|
+----+----------+----+----+----+
|null|  7.406317|  19| 1.0|null|
|null| -4.990888|   3| NaN|null|
|null|   8.07055|  15| NaN|null|
|null| -16.01117|   8| 3.0|null|
|null|-1.0873488|   8| 3.0|null|
+----+----------+----+----+----+



In [38]:
df.head(2)

[Row(id=5756852209, location=Row(altitude='104.9', country='UA', exact_location=0, id=22256, indoor=1, latitude='50.51', longitude='30.798'), sampling_rate=None, sensor=Row(id=36214, pin='7', sensor_type=Row(id=9, manufacturer='various', name='DHT22')), sensordatavalues=[Row(id=12224991603, value='10.00', value_type='temperature'), Row(id=12224991604, value='50.70', value_type='humidity')], timestamp='2019-12-13 11:10:02'),
 Row(id=5756852208, location=Row(altitude='111.8', country='GB', exact_location=1, id=21003, indoor=0, latitude='53.87869338867', longitude='-1.45841360092'), sampling_rate=None, sensor=Row(id=34792, pin='11', sensor_type=Row(id=17, manufacturer='Bosch', name='BME280')), sensordatavalues=[Row(id=12224991602, value='96357.16', value_type='pressure'), Row(id=12224991605, value='1.93', value_type='temperature'), Row(id=12224991606, value='100.00', value_type='humidity'), Row(id=None, value='97702.38', value_type='pressure_at_sealevel')], timestamp='2019-12-13 11:10:02'

In [39]:
df.filter(functions.col("timestamp").cast("date").name("Date") > "2019-01-01").show()

+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|        id|            location|sampling_rate|              sensor|    sensordatavalues|          timestamp|
+----------+--------------------+-------------+--------------------+--------------------+-------------------+
|5756852209|[104.9, UA, 0, 22...|         null|[36214, 7, [9, va...|[[12224991603, 10...|2019-12-13 11:10:02|
|5756852208|[111.8, GB, 1, 21...|         null|[34792, 11, [17, ...|[[12224991602, 96...|2019-12-13 11:10:02|
|5756852207|[20.5, BE, 0, 146...|         null|[16707, 1, [14, N...|[[12224991600, 9....|2019-12-13 11:10:02|
|5756852206|[499.5, DE, 0, 14...|         null|[2852, 1, [14, No...|[[12224991598, 0....|2019-12-13 11:10:02|
|5756852205|[54.2, BE, 0, 105...|         null|[20713, 1, [14, N...|[[12224991596, 0....|2019-12-13 11:10:02|
|5756852204|[78.1, RO, 0, 168...|         null|[29961, 7, [9, va...|[[12224991594, 5....|2019-12-13 11:10:02|
|575685220

### 删除数据
* 删除冗余数据 Spark 的 DataFrame 支持删除冗余数据，具有两种方法——[drop_duplicates()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop_duplicates) 和 [dropDuplicates()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates)，此外同 pandas 一样支持 subset 参数调整删除依据。需要注意的是对于流数据删除操作和实际操作具有差异性以避免删除后再重复的可能性，详情可以参考文档
* 删除缺失值 [pyspark.sql.DataFrame.drop_na()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropna) 和 [pyspark.sql.DataFrameNaFunctions.drop()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions.drop) 相同都是删除缺失值的方法，后者的使用方式是 `df.na.drop()`。同 pandas 一样支持 subset 参数调整。**需要注意，删除缺失值时，没有区分 NULL 以及 NAN 差别**
* 删除列数据 [pyspark.sql.DataFrame.drop()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) 支持直接删除列

In [40]:
# 删除冗余行
test.drop_duplicates(subset=["col1", "col5"]).count()

25

In [41]:
# 下面三者分别是是删除 col4 中的 NAN 以及 col1 中 NULL之后数量以及原始数量
test.dropna(subset=["col4"]).count(), test.dropna(subset=["col1"]).count(), test.count()

(23, 21, 30)

In [42]:
df.drop_duplicates().count()

35527

### 数据替换
* 缺失值替换 [pyspark.sql.DataFrame.fillna()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.fillna) 和 [pyspark.sql.DataFrameNaFunctions.fill()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions.fill) 相同都是替换缺失值的方法，后者的使用方式是 `df.na.fill()`。 NAN 和 NULL 上，均会被替代
* 指定数据替换 [pyspark.sql.DataFrame.replace()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.replace) 和 [pyspark.sql.DataFrameNaFunctions.replace()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions.replace)可以替换数据值。替换值的数据类型需要和原始数据类型相同
* 条件性替换 使用方式类似于 `case when`。 [`functions.when()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.when) 和 [`column.when()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.when) 存在差异，后者和 [`column.otherwise()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.otherwise) 相似是对前者的扩展。实际功能性上和 SQL 中 `case...[when...then...]else...` 类似

In [43]:
# 替换字符串
test.fillna("A").show()

+----+-----------+----+----+----+
|col1|       col2|col3|col4|col5|
+----+-----------+----+----+----+
|   T|-10.2372465|  18| 3.0|   A|
|   O|  1.5313411|   9| 0.0|   A|
|   S| -9.1716175|   5| 0.0|   J|
|   A|   7.406317|  19| 1.0|   A|
|   X|   5.094148|   3| NaN|   X|
|   E|  -3.525113|   3| 2.0|   A|
|   W|  -18.93121|  10| 2.0|   R|
|   A|  -4.990888|   3| NaN|   A|
|   T| -7.7867312|   2| 3.0|   A|
|   N| -14.736388|   6| 3.0|   A|
|   A|    8.07055|  15| NaN|   A|
|   A|  2.4262414|  19| 2.0|   K|
|   A| -4.2632003|   2| NaN|   V|
|   R| -3.1653242|   7| 0.0|   A|
|   S|  27.000383|   8| 1.0|   A|
|   A| -3.3306575|  10| 0.0|   E|
|   Z|  12.456934|  19| NaN|   K|
|   B| -3.7770996|  18| 2.0|   A|
|   A|  -16.01117|   8| 3.0|   A|
|   A| -1.0873488|   8| 3.0|   A|
+----+-----------+----+----+----+
only showing top 20 rows



In [44]:
test.fillna(12).show()

+----+-----------+----+----+----+
|col1|       col2|col3|col4|col5|
+----+-----------+----+----+----+
|   T|-10.2372465|  18| 3.0|null|
|   O|  1.5313411|   9| 0.0|null|
|   S| -9.1716175|   5| 0.0|   J|
|null|   7.406317|  19| 1.0|null|
|   X|   5.094148|   3|12.0|   X|
|   E|  -3.525113|   3| 2.0|null|
|   W|  -18.93121|  10| 2.0|   R|
|null|  -4.990888|   3|12.0|null|
|   T| -7.7867312|   2| 3.0|null|
|   N| -14.736388|   6| 3.0|null|
|null|    8.07055|  15|12.0|null|
|null|  2.4262414|  19| 2.0|   K|
|null| -4.2632003|   2|12.0|   V|
|   R| -3.1653242|   7| 0.0|null|
|   S|  27.000383|   8| 1.0|null|
|null| -3.3306575|  10| 0.0|   E|
|   Z|  12.456934|  19|12.0|   K|
|   B| -3.7770996|  18| 2.0|null|
|null|  -16.01117|   8| 3.0|null|
|null| -1.0873488|   8| 3.0|null|
+----+-----------+----+----+----+
only showing top 20 rows



### 其他操作
* 字符拼接 需要使用 `functions.concat()` 或者 `functions.concat_ws()` 可以完成相关任务，后者可以指定拼接间隔符
* 日期计算 使用 `functions.date_add()`、`functions.date_sub()`——日期加减 `functions.datediff()`——日期间间隔天数，`functions.date_trunc()`——提取时间戳及日期中年月日以及时间点，可以试用格式 ‘year’, ‘yyyy’, ‘yy’, ‘month’, ‘mon’, ‘mm’, ‘day’, ‘dd’, ‘hour’, ‘minute’, ‘second’, ‘week’, ‘quarter’。其他类似于 Pandas 中 TimeStamp 的 Series 的属性提取操作，使用 `functions.dayofmonth()` 等

## 参考
1. [Unlike Pandas, PySpark doesn’t consider NaN values to be NULL. See the NaN Semantics](https://spark.apache.org/docs/latest/sql-reference.html#nan-semantics)
2. [ Spark SQL, Built-in Functions](https://spark.apache.org/docs/latest/api/sql/)


在调整数据过程中需要注意是否有必要进行原位替换，还是只需要部分数据——需要判断是使用 `withColumn()` 还是 `select()`

: org.apache.spark.sql.catalyst.parser.ParseException: 
extraneous input ''' expecting {'(', 'SELECT', 'FROM', 'ADD', 'AS', 'ALL', 'ANY', 'DISTINCT', 'WHERE', 'GROUP', 'BY', 'GROUPING', 'SETS', 'CUBE', 'ROLLUP', 'ORDER', 'HAVING', 'LIMIT', 'AT', 'OR', 'AND', 'IN', NOT, 'NO', 'EXISTS', 'BETWEEN', 'LIKE', RLIKE, 'IS', 'NULL', 'TRUE', 'FALSE', 'NULLS', 'ASC', 'DESC', 'FOR', 'INTERVAL', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'JOIN', 'CROSS', 'OUTER', 'INNER', 'LEFT', 'SEMI', 'RIGHT', 'FULL', 'NATURAL', 'ON', 'PIVOT', 'LATERAL', 'WINDOW', 'OVER', 'PARTITION', 'RANGE', 'ROWS', 'UNBOUNDED', 'PRECEDING', 'FOLLOWING', 'CURRENT', 'FIRST', 'AFTER', 'LAST', 'ROW', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'DIRECTORY', 'VIEW', 'REPLACE', 'INSERT', 'DELETE', 'INTO', 'DESCRIBE', 'EXPLAIN', 'FORMAT', 'LOGICAL', 'CODEGEN', 'COST', 'CAST', 'SHOW', 'TABLES', 'COLUMNS', 'COLUMN', 'USE', 'PARTITIONS', 'FUNCTIONS', 'DROP', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'TO', 'TABLESAMPLE', 'STRATIFY', 'ALTER', 'RENAME', 'ARRAY', 'MAP', 'STRUCT', 'COMMENT', 'SET', 'RESET', 'DATA', 'START', 'TRANSACTION', 'COMMIT', 'ROLLBACK', 'MACRO', 'IGNORE', 'BOTH', 'LEADING', 'TRAILING', 'IF', 'POSITION', 'EXTRACT', '+', '-', '*', 'DIV', '~', 'PERCENT', 'BUCKET', 'OUT', 'OF', 'SORT', 'CLUSTER', 'DISTRIBUTE', 'OVERWRITE', 'TRANSFORM', 'REDUCE', 'SERDE', 'SERDEPROPERTIES', 'RECORDREADER', 'RECORDWRITER', 'DELIMITED', 'FIELDS', 'TERMINATED', 'COLLECTION', 'ITEMS', 'KEYS', 'ESCAPED', 'LINES', 'SEPARATED', 'FUNCTION', 'EXTENDED', 'REFRESH', 'CLEAR', 'CACHE', 'UNCACHE', 'LAZY', 'FORMATTED', 'GLOBAL', TEMPORARY, 'OPTIONS', 'UNSET', 'TBLPROPERTIES', 'DBPROPERTIES', 'BUCKETS', 'SKEWED', 'STORED', 'DIRECTORIES', 'LOCATION', 'EXCHANGE', 'ARCHIVE', 'UNARCHIVE', 'FILEFORMAT', 'TOUCH', 'COMPACT', 'CONCATENATE', 'CHANGE', 'CASCADE', 'RESTRICT', 'CLUSTERED', 'SORTED', 'PURGE', 'INPUTFORMAT', 'OUTPUTFORMAT', DATABASE, DATABASES, 'DFS', 'TRUNCATE', 'ANALYZE', 'COMPUTE', 'LIST', 'STATISTICS', 'PARTITIONED', 'EXTERNAL', 'DEFINED', 'REVOKE', 'GRANT', 'LOCK', 'UNLOCK', 'MSCK', 'REPAIR', 'RECOVER', 'EXPORT', 'IMPORT', 'LOAD', 'ROLE', 'ROLES', 'COMPACTIONS', 'PRINCIPALS', 'TRANSACTIONS', 'INDEX', 'INDEXES', 'LOCKS', 'OPTION', 'ANTI', 'LOCAL', 'INPATH', STRING, BIGINT_LITERAL, SMALLINT_LITERAL, TINYINT_LITERAL, INTEGER_VALUE, DECIMAL_VALUE, DOUBLE_LITERAL, BIGDECIMAL_LITERAL, IDENTIFIER, BACKQUOTED_IDENTIFIER}(line 1, pos 8)